In [ ]:
import pandas_datareader as pdr
import pandas as pd
# k = "1dceb00b00eba08f93b9440b08f94bac5f21583b"

In [ ]:
# data  = pdr.get_data_tiingo('RYSOX',api_key=k)
# data.to_csv('/content/sample_data/S&P.csv')
data = pd.read_csv('/content/sample_data/S&P.csv')
data

In [ ]:
df1 = data.reset_index()['close']
df1

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(df1)

In [ ]:
import numpy as np

In [ ]:
from sklearn.preprocessing import MinMaxScaler ##Lstm Sensitive to scale of data
scaler = MinMaxScaler(feature_range=(0,1)) ## values b/w 0 to 1
df1 = scaler.fit_transform(np.array(df1).reshape(-1,1))
df1.shape

In [ ]:
# train data and test data allocation

training_size=int(len(df1)*0.80)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

In [ ]:
training_size,test_size

In [ ]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [ ]:
print(X_test.shape)
print(ytest.shape)

In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

In [ ]:
import tensorflow as tf

In [ ]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
##Transformback to original form
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
rmse_train_lstm = math.sqrt(mean_squared_error(y_train,train_predict))
rmse_train_lstm

In [ ]:
### Test Data RMSE
rmse_test_lstm = math.sqrt(mean_squared_error(ytest,test_predict))
rmse_test_lstm

In [ ]:
import matplotlib.pyplot as plt
look_back=100
trainPredictPlot = numpy.empty_like(df1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(df1)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1, :] = test_predict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(df1))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
len(test_data)

In [ ]:
x_input=test_data[152:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()
temp_input

In [ ]:
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<30):

    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1


print(lst_output)

In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,131)

In [ ]:
len(df1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(day_new,scaler.inverse_transform(df1[1157:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))

In [ ]:
df3=df1.tolist()
df3.extend(lst_output)
plt.plot(df3[1200:])

In [ ]:
df3=scaler.inverse_transform(df3).tolist()

In [ ]:
plt.plot(df3)

# **LSTM + Dense Nureal**

In [ ]:
model_combined = Sequential()
model_combined.add(LSTM(50, return_sequences=True, input_shape=(100, 1)))
model_combined.add(LSTM(50, return_sequences=True))
model_combined.add(LSTM(50))
model_combined.add(Dense(50, activation='relu'))  # Add a dense layer
model_combined.add(Dense(1))
model_combined.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model_combined.fit(X_train, y_train, validation_data=(X_test, ytest), epochs=100, batch_size=64, verbose=1)

In [ ]:
train_predict_combined = model_combined.predict(X_train)
test_predict_combined = model_combined.predict(X_test)

In [ ]:
train_predict_combined = scaler.inverse_transform(train_predict_combined)
test_predict_combined = scaler.inverse_transform(test_predict_combined)

In [ ]:
rmse_train_combined = np.sqrt(mean_squared_error(y_train, train_predict_combined))
rmse_test_combined = np.sqrt(mean_squared_error(ytest, test_predict_combined))

In [ ]:
print(rmse_train_combined)
print(rmse_test_combined)

# **Stacked LSTMs with Regularization**

In [ ]:
from tensorflow.keras.layers import Dropout

# Define the LSTM model with dropout layers
model_lstm_dropout = Sequential()
model_lstm_dropout.add(LSTM(50, return_sequences=True, input_shape=(100, 1)))
model_lstm_dropout.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model_lstm_dropout.add(LSTM(50, return_sequences=True))
model_lstm_dropout.add(Dropout(0.2))
model_lstm_dropout.add(LSTM(50))
model_lstm_dropout.add(Dense(1))
model_lstm_dropout.compile(loss='mean_squared_error', optimizer='adam')


In [ ]:
model_lstm_dropout.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

In [ ]:
train_predict_dropout = model_lstm_dropout.predict(X_train)
test_predict_dropout = model_lstm_dropout.predict(X_test)

In [ ]:
### Calculate RMSE performance metrics
train_predict_dropout=scaler.inverse_transform(train_predict_dropout)
test_predict_dropout=scaler.inverse_transform(test_predict_dropout)


In [ ]:
import math
from sklearn.metrics import mean_squared_error
rmse_train_lstm_drop = math.sqrt(mean_squared_error(y_train,train_predict_dropout))
print(rmse_train_lstm_drop)
rmse_test_lstm_drop = math.sqrt(mean_squared_error(ytest,test_predict_dropout))
print(rmse_test_lstm_drop)

# **CNN-LSTM Hybrid**

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D

# Define a CNN-LSTM hybrid model
model_cnn_lstm = Sequential()
model_cnn_lstm.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(100, 1)))
model_cnn_lstm.add(MaxPooling1D(pool_size=2))
model_cnn_lstm.add(LSTM(50, return_sequences=True))
model_cnn_lstm.add(LSTM(50))
model_cnn_lstm.add(Dense(1))
model_cnn_lstm.compile(loss='mean_squared_error', optimizer='adam')

# Train and evaluate the model as before


In [ ]:
model_cnn_lstm.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

In [ ]:
train_predict_cnn = model_cnn_lstm.predict(X_train)
test_predict_cnn = model_cnn_lstm.predict(X_test)

In [ ]:
train_predict_cnn = scaler.inverse_transform(train_predict_cnn)
test_predict_cnn = scaler.inverse_transform(test_predict_cnn)

In [ ]:
rmse_train_cnn = np.sqrt(mean_squared_error(y_train, train_predict_cnn))
rmse_test_cnn = np.sqrt(mean_squared_error(ytest, test_predict_cnn))

In [ ]:
print(rmse_train_cnn)
print(rmse_test_cnn)

In [ ]:
print("LSTM",rmse_test_lstm)
print("\nLSTM + Dense nureal : ",rmse_test_combined)
print("\nLSTM + Regularization : ",rmse_test_lstm_drop)
print("\nLSTM-CNN Hybride : ",rmse_test_cnn)


In [ ]:
from tensorflow.keras.layers import GRU

model_gru = Sequential()
model_gru.add(GRU(50, return_sequences=True, input_shape=(100, 1)))
model_gru.add(GRU(50, return_sequences=True))
model_gru.add(GRU(50))
model_gru.add(Dense(1))
model_gru.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model_gru.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

In [ ]:
train_predict_gru = model_gru.predict(X_train)
test_predict_gru = model_gru.predict(X_test)

In [ ]:
rmse_train_gru = np.sqrt(mean_squared_error(y_train, train_predict_gru))
rmse_test_gru = np.sqrt(mean_squared_error(ytest, test_predict_gru))

In [ ]:
print(rmse_train_gru)
print(rmse_test_gru)

In [ ]:
model_lstm_gru = Sequential()
model_lstm_gru.add(LSTM(50, return_sequences=True, input_shape=(100, 1)))
model_lstm_gru.add(LSTM(50, return_sequences=True))
model_lstm_gru.add(GRU(50, return_sequences=True))  # Adding a GRU layer
model_lstm_gru.add(GRU(50))
model_lstm_gru.add(Dense(1))
model_lstm_gru.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model_lstm_gru.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

In [ ]:
train_predict_lstm_gru = model_lstm_gru.predict(X_train)
test_predict_lstm_gru = model_lstm_gru.predict(X_test)

In [ ]:
rmse_train_lstm_gru = np.sqrt(mean_squared_error(y_train, train_predict_lstm_gru))
rmse_test_lstm_gru = np.sqrt(mean_squared_error(ytest, test_predict_lstm_gru))

In [ ]:
print(rmse_train_lstm_gru)
print(rmse_test_lstm_gru)

In [ ]:
print("LSTM : ",rmse_test_lstm)
print("\nLSTM + Dense nureal : ",rmse_test_combined)
print("\nLSTM + Regularization : ",rmse_test_lstm_drop)
print("\nLSTM-CNN Hybride : ",rmse_test_cnn)
print("\nGRU : ",rmse_test_gru)
print("\nLSTM + GRU : ",rmse_test_lstm_gru)